In [966]:
import matplotlib.pyplot as plt

In [200]:
from docscorer.configuration import ScorerConfiguration
from docscorer.docscorer import DocumentScorer
import json
import pandas as pd
config = ScorerConfiguration()
scorer = DocumentScorer(config)

path = "/mnt/d/Prompsit/WDS/aarepo_github/datos/ja_language.jsonl"
documents = []
i = 0
with open(path, "r", encoding="utf8") as a:
    for document in a:
        if i == 10000:
            break
        documents.append(json.loads(document))
        i+=1

In [3]:
from typing import Any
def get_threshold(
    table: dict[str, Any], language: str, default_key: str = "standard"
) -> Any:
    """Fetch a language-specific threshold, falling back to `default_key`."""
    try:
        return table.get(language, table[default_key])
    except KeyError:
        raise KeyError(
            f"Neither '{language}' nor default '{default_key}' found in {table}."
        ) from None


def scale_value(
    value: float,
    min_value: float,
    max_value: float,
    min_score: float,
    max_score: float,
) -> float:
    """Scale a value linearly into a score range."""
    if min_value == max_value:
        return 0.0

    ratio = (value - min_value) / (max_value - min_value)
    score = ratio * (max_score - min_score) + min_score
    return score

In [201]:
mesured_docs = [ (scorer._extract_features(document_text=x["text"]), x["text"]) for x in documents]

In [20]:
class NumsScorer:
    def __init__(self, config: ScorerConfiguration):
        self.config = config

    def score(self, ref_language: str, num_numbers: int, num_word_chars: int) -> float:
        if num_word_chars == 0:
            return 0.0

        percent_max = get_threshold(self.config.NUMBERS_PERCENT_MAX, ref_language)
        percent_bad = get_threshold(self.config.NUMBERS_PERCENT_BAD, ref_language)
        percent_semibad = get_threshold(
            self.config.NUMBERS_PERCENT_SEMIBAD, ref_language
        )
        percent_desired = get_threshold(
            self.config.NUMBERS_PERCENT_DESIRED, ref_language
        )

        ratio = round((num_numbers / num_word_chars) * 100, 1)

        if ratio <= percent_desired:
            return 1.0
        if ratio >= percent_bad:
            ratio = min(ratio, percent_max)
            return scale_value(ratio, percent_max, percent_bad, 0.0, 0.5)
        if ratio >= percent_semibad:
            return scale_value(ratio, percent_bad, percent_semibad, 0.5, 0.7)
        return scale_value(ratio, percent_semibad, percent_desired, 0.7, 1.0)

In [9]:
numbers_scorer = NumsScorer(config)

In [24]:
numbers_score=numbers_scorer.score("ita_latn", sum(doc[0]["numbers"]), sum(doc[0]["word_chars"]))
numbers_score

0.9213592233009709

In [121]:
config.NUMBERS_PERCENT_MAX["ita_latn"]

34.6

In [187]:
def penalize_lack_punct_segm(punct_chars: list, word_chars: list, not_penalized: float) -> float:
    total_word = sum(word_chars)
    if not total_word:
        return 0, 0
    bad_segm = 0
    for n in range(len(punct_chars)):
        n_word_ch = word_chars[n]
        n_analyzed = punct_chars[n]
        if not n_word_ch or n_word_ch <= not_penalized:
            continue
        ratio = round((n_analyzed / n_word_ch) * 100, 1)
        
        if ratio < percent_bad:
            bad_segm += n_word_ch
    proportion_bad = bad_segm/total_word
    if proportion_bad < 0.05:
        return 1, proportion_bad
    elif proportion_bad > 0.4:
        return 0, proportion_bad
    elif proportion_bad > 0.05 and proportion_bad < 0.2:
        return scale_value(proportion_bad, 0.2, 0.05, 0.5, 1.0), proportion_bad
    else:
        return scale_value(proportion_bad, 0.4, 0.2, 0, 0.5), proportion_bad
    
        

In [211]:
ref_language = "jpn_jpan"
percent_bad = get_threshold(config.PUNCTUATION_PERCENT_BAD, ref_language)[1]
long_text_min = get_threshold(config.LONG_TEXT_MIN, ref_language)
long_text_max = get_threshold(config.LONG_TEXT_MAX, ref_language)
menu_length = get_threshold(config.MENUS_AVERAGE_LENGTH, ref_language)

# n = 84
for n in range(35+1, len(mesured_docs)):
    data = mesured_docs[n][0]
    text = mesured_docs[n][1]
    penal = penalize_lack_punct_segm(punct_chars = data["punctuation_chars"], word_chars = data["word_chars"], 
                                not_penalized = menu_length*3)
    if penal[0] <0.8:
        print(n)
        print(penal)
        print(text)
        break

52
(0.4324391455538997, 0.22702434177844014)
2020年夏・トスカーナの風景~その3・オルチャ渓谷の乾いた大地 | トスカーナ自由自在
トスカーナでの旅・取材・視察・ビジネスなら
検索 menu
ごあいさつ
サービスメニュー
お問い合わせ
and
or
目的から選ぶ トスカーナ自由自在チャンネル 都市 小さな村 お祭り・イベント 郷土料理・食材 体験プログラム    ワイナリー    ウェディング    料理教室 ハイヤー&アテンド周遊プラン 見る 泊まる 食べる    レストラン    バール    ジェラテリア 買う 移動する コラム 未分類
エリアから選ぶ フィレンツェ県    フィレンツェ    キアンティ    ムジェッロ&シエーヴェ渓谷 シエナ県    クレーテ・セネージ    キアーナ渓谷    シエナ    オルチャ渓谷    アミアータ山周辺 ピサ県    ピサ ルッカ県    ルッカ    ガルファニャーナ アレッツォ県    アルノ渓谷    カゼンティーノ    アレッツォ    ティベリーナ渓谷    キアーナ渓谷 グロッセート県    アミアータ山周辺    グロッセート    マレンマ    ジリオ島 リヴォルノ県    エルバ島    リヴォルノ    エトルリア海岸 マッサ・カッラーラ県 ピストイア県    ピストイア プラート県    プラート トスカーナ以外    マルケ    ウンブリア    プーリア    フリウリ・ヴェネツィア・ジュリア    エミリア・ロマーニャ    ヴェネト    ロンバルディア    ラツィオ    リグーリア    アブルッツォ    トレンティーノ・アルトアディジェ イタリア以外    フランス    ギリシャ
ホーム
トスカーナ自由自在
見る
シエナ県
オルチャ渓谷
2020年夏・トスカーナの風景~その3・オルチャ渓谷の乾いた大地
見る
シエナ県オルチャ渓谷
2020年夏・トスカーナの風景~その3・オルチャ渓谷の乾いた大地
2020.08.18
日本もイタリアも、毎日暑いですね〜。おらが村、ばーちゃん村は山手のため、いつも身近に緑がありますが、平地・丘陵地帯は草刈りや土起こしの後なので、からっからの茶色の大地が広がっています。
車窓から見るオルチャ

In [212]:
n =52
data = mesured_docs[n][0]
text = mesured_docs[n][1].split("\n")
punct_chars = data["punctuation_chars"]
word_chars = data["word_chars"]
for i in range(len(punct_chars)):
    n_word_ch = word_chars[i]
    n_analyzed = punct_chars[i]
    if not n_word_ch or n_word_ch <= menu_length*3:
        continue
    ratio = round((n_analyzed / n_word_ch) * 100, 1)
    
    if ratio < percent_bad:
        print(text[i])

2020年夏・トスカーナの風景~その3・オルチャ渓谷の乾いた大地 | トスカーナ自由自在
目的から選ぶ トスカーナ自由自在チャンネル 都市 小さな村 お祭り・イベント 郷土料理・食材 体験プログラム    ワイナリー    ウェディング    料理教室 ハイヤー&アテンド周遊プラン 見る 泊まる 食べる    レストラン    バール    ジェラテリア 買う 移動する コラム 未分類
エリアから選ぶ フィレンツェ県    フィレンツェ    キアンティ    ムジェッロ&シエーヴェ渓谷 シエナ県    クレーテ・セネージ    キアーナ渓谷    シエナ    オルチャ渓谷    アミアータ山周辺 ピサ県    ピサ ルッカ県    ルッカ    ガルファニャーナ アレッツォ県    アルノ渓谷    カゼンティーノ    アレッツォ    ティベリーナ渓谷    キアーナ渓谷 グロッセート県    アミアータ山周辺    グロッセート    マレンマ    ジリオ島 リヴォルノ県    エルバ島    リヴォルノ    エトルリア海岸 マッサ・カッラーラ県 ピストイア県    ピストイア プラート県    プラート トスカーナ以外    マルケ    ウンブリア    プーリア    フリウリ・ヴェネツィア・ジュリア    エミリア・ロマーニャ    ヴェネト    ロンバルディア    ラツィオ    リグーリア    アブルッツォ    トレンティーノ・アルトアディジェ イタリア以外    フランス    ギリシャ


In [57]:
 ਸਾਡੇ ਦੁਆਰਾ ਜੋ ਵੀ ਅਪਡੇਟ ਤੇ ਵਾਇਰਲ ਖਬਰ ਅਤੇ ਘਰੇਲੂ ਨੁਸਖੇ ਦਿੱਤੇ ਜਾਣਗੇ ਉਹ ਤੁਹਾਡੇ ਤੱਕ ਸਭ ਤੋਂ ਪਹਿਲਾਂ ਪਹੁੰਚ ਜਾਣਗੇ ਤੇ ਤੁਹਾਨੂੰ ਇੱਕ ਚੰਗੀ ਤੇ ਫਾਇਦੇਮੰਦ ਜਾਣਕਾਰੀ ਮਿਲੇਗੀ |ਇਸ ਕਰਕੇ ਸਾਰੇ ਵੀਰਾਂ ਭੈਣਾਂ ਨੂੰ ਬੇਨਤੀ ਹੈ ਕਿ ਜਿੰਨਾਂ ਵੀਰਾਂ ਨੇ ਸਾਡੇ ਪੇਜ ਨੂੰ ਲਾਇਕ ਨਹੀਂ ਕੀਤਾ ਉਹ ਪੇਜ ਨੂੰ ਲਾਇਕ ਕਰੋ ਤੇ ਜਿੰਨਾਂ ਵੀਰਾਂ ਨੂੰ ਪੇਜ ਨੂੰ ਲਾਇਕ ਕੀਤਾ ਹੋਇਆ ਹੈ ਉਹਨਾਂ ਦਾ ਦਿਲੋਂ ਧੰਨਵਾਦ ਹੈ ਜੀ |ਸਾਡੀ ਹਰ ਵੇਲੇ ਏਹੀ ਕੋਸ਼ਿਸ਼ ਹੁੰਦੀ ਹੈ ਕਿ ਤੁਹਾਡੇ ਤੱਕ ਸਿਰਫ਼ ਸੱਚ ਤੇ ਸਟੀਕ ਜਾਣਕਾਰੀ ਹੀ ਮਹੁੱਈਆ ਕਰਵਾਈ ਜਾਵੇ ਤਾਂ ਜੋ ਤੁਸੀਂ ਉਸਨੂੰ ਆਪਣੀ ਨਿੱਜੀ ਜਿੰਦਗੀ ਦੇ ਵਿਚ ਚੰਗੀ ਤਰਾਂ ਫੋਲੋ ਕਰ ਸਕੋਂ ਤੇ ਉਸ ਤੋਂ ਫਾਇਦਾ ਲੈ ਸਕੋਂ ਤੇ ਇੱਕ ਚੰਗੀ ਜੀਵਨਸ਼ੈਲੀ ਬਤੀਤ ਕਰ ਸਕੋਂ |

0.6

In [58]:
ratio

0.5

In [ ]:
def penalize_lack_punct_segm(punct_chars: list, word_chars: list, not_penalized: 
                          int, mid_penalized: int, hard_penalized: int, percent_semibad) -> float:
    worst_segm = 0
    for n in range(len(punct_chars)):
        n_word_ch = word_chars[n]
        n_analyzed = punct_chars[n]
        if not n_word_ch or n_word_ch <= not_penalized:
            continue
        ratio = round((n_analyzed / n_word_ch) * 100, 1)
        
        if ratio < percent_semibad:
            # worst_segm = n_word_ch if n_word_ch > worst_segm else worst_segm
            worst_segm += n_word_ch
    if worst_segm <= not_penalized:
        return 1.0, 0
    elif worst_segm > hard_penalized:
        return 0.0, worst_segm
    elif worst_segm <= mid_penalized:
        return scale_value(worst_segm, mid_penalized, not_penalized, 0.5, 1.0), worst_segm
    return scale_value(worst_segm, hard_penalized, mid_penalized, 0.0, 0.5), worst_segm

In [ ]:
self, ref_language: str, num_punctuation_chars: int, num_word_chars: int, punct_chars: list, 
        word_chars: list, menu_lenth: int